In [1]:
import numpy as np
import pandas as pd
import xarray as xr

In [19]:
pd.DataFrame({'longitude': dwd_Hornsea1.longitude.values, 'latitude': dwd_Hornsea1.latitude.values, }).to_csv('dwd_coords_Hornsea1.csv', index=False)

In [2]:
dwd_Hornsea1 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc")
dwd_Hornsea1_features = dwd_Hornsea1["WindSpeed:100"].mean(dim=["latitude","longitude"]).to_dataframe().reset_index()
dwd_Hornsea1_features["ref_datetime"] = dwd_Hornsea1_features["ref_datetime"].dt.tz_localize("UTC")
dwd_Hornsea1_features["valid_datetime"] = dwd_Hornsea1_features["ref_datetime"] + pd.TimedeltaIndex(dwd_Hornsea1_features["valid_datetime"],unit="hours")

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_74869/835483287.py:4: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  dwd_Hornsea1_features["valid_datetime"] = dwd_Hornsea1_features["ref_datetime"] + pd.TimedeltaIndex(dwd_Hornsea1_features["valid_datetime"],unit="hours")


In [22]:
pd.DataFrame({'longitude': dwd_solar.longitude.values, 'latitude': dwd_solar.latitude.values, }).to_csv('dwd_coords_solar.csv', index=False)

In [21]:
dwd_solar = xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20231027.nc")
dwd_solar_features = dwd_solar["SolarDownwardRadiation"].mean(dim="point").to_dataframe().reset_index()
dwd_solar_features["ref_datetime"] = dwd_solar_features["ref_datetime"].dt.tz_localize("UTC")
dwd_solar_features["valid_datetime"] = dwd_solar_features["ref_datetime"] + pd.TimedeltaIndex(dwd_solar_features["valid_datetime"],unit="hours")

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_74869/167480448.py:4: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  dwd_solar_features["valid_datetime"] = dwd_solar_features["ref_datetime"] + pd.TimedeltaIndex(dwd_solar_features["valid_datetime"],unit="hours")


In [4]:
energy_data = pd.read_csv("data/Energy_Data_20200920_20231027.csv")
energy_data["dtm"] = pd.to_datetime(energy_data["dtm"])
energy_data["Wind_MWh_credit"] = 0.5*energy_data["Wind_MW"] - energy_data["boa_MWh"]
energy_data["Solar_MWh_credit"] = 0.5*energy_data["Solar_MW"]

In [168]:
data = dwd_Hornsea1_features.merge(dwd_solar_features,how="outer",on=["ref_datetime","valid_datetime"])
data = data.set_index("valid_datetime").groupby("ref_datetime").resample("30T").interpolate("linear")
data = data.drop(columns="ref_datetime",axis=1).reset_index()
data = data.merge(energy_data,how="inner",left_on="valid_datetime",right_on="dtm")
data = data[data["valid_datetime"] - data["ref_datetime"] < np.timedelta64(50,"h")]
data.rename(columns={"WindSpeed:100":"WindSpeed"}, inplace=True)
data["total_generation_MWh"] = data["Wind_MWh_credit"] + data["Solar_MWh_credit"]
data.drop(columns=["dtm"], inplace=True)

/var/folders/sj/xlrs69b568n4mntwz_7crr4r0000gn/T/ipykernel_65470/2776683689.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data = data.set_index("valid_datetime").groupby("ref_datetime").resample("30T").interpolate("linear")


In [169]:
condition1 = data['ref_datetime'].dt.hour == 0

condition2 = ((data['valid_datetime'].dt.hour >= 23) & (data['valid_datetime'].dt.date == data['ref_datetime'].dt.date)) | \
             ((data['valid_datetime'].dt.hour < 23) & (data['valid_datetime'].dt.date == (data['ref_datetime'] + pd.Timedelta(days=1)).dt.date))

data = data[condition1 & condition2]
data = data.iloc[:-2]
data = data.reset_index(drop=True)
data["market_day"] = data["ref_datetime"].dt.date+pd.Timedelta(days=1)

In [171]:
data.to_csv("data/data_heftcom2024.csv", index=False)

In [163]:
# What to do now? 
# 0) Data can always be messy. We should always clean data first before creating the environment. Right? 
# 1) save the data to a csv file
# 2) Add the data to the environment
# 3) Loop through the market days and feed the state
# 4) Run the Agent. 

In [ ]:
prediction = model.predict()
prediction.shape = (samples, horizon, targets)
target 

error = (prediction - target)**2
plt.plot(error.mean(axis=0)) 


class Predictor(rb.Predictor): 
    def __init__(self, windfarm: rb.WindFarm):
        self.windfarm = windfarm
        self.model = 

    def laod_training_data(): 

    def load_predict_data(): 

    def train(): 

        self.model =

    def predict(self, state: rb.Input, horizon: int) -> np.ndarray:


# 1) create rb.WindFarm
# 2) create rb.Predictor 
# 3) train on training data (optional RWE API otherwise csv-file)
# 4) evaluate train and test and plot in notebook
# 5) deploy model using toolkit
# 6) use model API to predict on lastest data
# 7) plot the last forecast in notebook

